In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Define a function    

In [3]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

### The Functions List

In [8]:
#OpenAI added a parameter for functions with is a list of JSON objects
#NOTE: the description fields are very important.  They are what is passed to the LLM.
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
### The Messages List

In [6]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [9]:
import openai

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

NOTE: the response return an object with no "content", but it does prepare to call our function (based on the weather related question we asked)

In [10]:
print(response)

{
  "id": "chatcmpl-8FArlWYHci1wSTHMKDFwT8kQorvJz",
  "object": "chat.completion",
  "created": 1698627381,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n\"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 17,
    "total_tokens": 99
  }
}


In [12]:
#the "arguments" field is itself a JSON object that can be passed directly to our function
#NOTE: LangChain won't call our function directly
response_message = response["choices"][0]["message"]
args = json.loads(response_message["function_call"]["arguments"])
print(f"args = {args}")
get_current_weather(args)

args = {'location': 'Boston, MA'}


'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

___
If we don't ask a weather related question, the "content" will be populated in the response and the function won't get returned.

In [13]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)
print(response)

{
  "id": "chatcmpl-8FB044l7jfKIzRTBv1pIqRKbjEnMi",
  "object": "chat.completion",
  "created": 1698627896,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


### ```function_call``` parameter
3 valid values
- ```auto```: the default
- ```none```: always ignore the functions
- pass the function: this would always try to create params for that function

In [15]:
#NONE - the prompt is stated such that it could use our function, but it doens't because of the NONE param
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8FB55NM9ynYVd2CB4h4QEg3MMFGcY",
  "object": "chat.completion",
  "created": 1698628207,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Let me check the current weather in Boston for you."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 11,
    "total_tokens": 93
  }
}


In [16]:
#FORCE - Our prompt is not related to the weather, but we still get a response as if it does, because we are forcing, given the "function_call" param
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8FB6pqOywA08fOjItzEeZB64hLDQK",
  "object": "chat.completion",
  "created": 1698628315,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  }
}


### Incorporate function response to the conversation

In [17]:
#After the chain responds with function params
#I need to call the function myself
#then they suggest to include the response from the function (that I called manually) to the messages and call the chat again.
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8FBQmsmFXVtUwe8tuGfLhrFCtgt3Z",
  "object": "chat.completion",
  "created": 1698629552,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n\"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 10,
    "total_tokens": 99
  }
}


In [18]:
#append the ASSISTANT's response
messages.append(response["choices"][0]["message"])

In [20]:
#manually call the function and capture that response as "observation"
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [ ]:
#append that "observation" as the "function" role to the messages
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [22]:
#Enter that back into the conversation, by calling chat with the messages array
#It will take our obseravation in object form and respond with natural language
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-8FBTwudCneKgM5dHO6xP1aQF4J0Ph",
  "object": "chat.completion",
  "created": 1698629748,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is partly cloudy with a temperature of 62\u00b0F (17\u00b0C) and winds blowing at 9 mph."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 35,
    "completion_tokens": 29,
    "total_tokens": 64
  }
}
